In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Add pre-installed pytorch3d to sys.path
import sys
sys.path.append("/content/drive/My Drive/GoogleColab/pytorch3d_packages")

In [ ]:
import os
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from pytorch3d.datasets import ShapeNetCore


# Unzipping the dataset
# Note, problem with the code. For extract_path, it needs to run once and get an error,
# then change the variable to extract_path = "/content/ShapeNetCore/ShapeNetCore/ShapeNetCore"

zip_path = "/content/drive/My Drive/GoogleColab/ShapeNetCore.zip"
extract_path = "/content/ShapeNetCore/ShapeNetCore"

if not os.path.exists(extract_path):
    print("Extracting dataset...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(extract_path))
    print("Extraction complete.")
else:
    print("Dataset already extracted.")

# Check to see that the ShapeNetCore models are found
print(os.listdir(extract_path))


# Variables
categories = ["03642806", "03211117", "03046257", "02992529", "02808440"]
category_names = ["Laptop", "Monitor", "Clock", "Cellphone", "Bathtub"]
categories_to_idx = {s: i for i, s in enumerate(categories)}
batch_size = 16
lr = 0.001
epochs = 20
num_points = 500   # fixed input size for point clouds
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print("Using device:", device)


# Collate function that converts a batch of ShapeNet samples into fixed-size point clouds with labels
def collate_pointcloud(batch):
    verts_list = [b["verts"] for b in batch]
    synset_ids = [b["synset_id"] for b in batch]
    labels = torch.tensor([categories_to_idx[sid] for sid in synset_ids], dtype=torch.long)

    pcs = []
    for v in verts_list:
        n = v.shape[0]
        if n >= num_points: # Check if we need to add/remove points
            idx = torch.randperm(n)[:num_points]
            pc = v[idx]
        else:
            pad = torch.zeros(num_points - n, 3)
            pc = torch.cat([v, pad], dim=0)
        pcs.append(pc)

    pcs = torch.stack(pcs, dim=0)
    return pcs, labels


# Point Cloud classifier
class PointNetClassifier(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()
        self.mlp1 = nn.Linear(3, 64)
        self.mlp2 = nn.Linear(64, 128)
        self.mlp3 = nn.Linear(128, 256)
        self.fc1 = nn.Linear(256, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.mlp1(x))
        x = F.relu(self.mlp2(x))
        x = F.relu(self.mlp3(x))
        x = torch.max(x, dim=1)[0]
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


# Getting the dataset
dataset = ShapeNetCore(
    data_dir=extract_path,
    synsets=categories,
    version=2,
    load_textures=False
)
print("Total models loaded:", len(dataset))

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_pointcloud, num_workers=0)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_pointcloud, num_workers=0)


# Training the model
model = PointNetClassifier(num_classes=len(categories)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for pcs, labels in train_loader:
        pcs, labels = pcs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(pcs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    correct, total = 0, 0
    class_correct = [0]*len(categories)
    class_total = [0]*len(categories)
    with torch.no_grad():
        for pcs, labels in val_loader:
            pcs, labels = pcs.to(device), labels.to(device)
            outputs = model(pcs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            for i in range(labels.size(0)):
                gt = labels[i].item()
                pred = predicted[i].item()
                class_total[gt] += 1
                if gt == pred:
                    class_correct[gt] += 1

    val_acc = correct / total if total > 0 else 0.0
    avg_loss = total_loss / (len(train_loader) if len(train_loader) > 0 else 1)
    print(f"Epoch {epoch+1}/{epochs}  Loss: {avg_loss:.4f}  Val Acc: {val_acc:.4f}")
    for i, name in enumerate(category_names):
        if class_total[i] > 0:
            print(f"  {name}: {class_correct[i]/class_total[i]:.4f}")
    print()


Extracting dataset...
Extraction complete.
['02808440', '03642806', '02992529', '03211117', '03046257']
Using device: cuda


/content/drive/My Drive/GoogleColab/pytorch3d_packages/pytorch3d/datasets/shapenet/shapenet_core.py:116: UserWarning: The following categories are included in ShapeNetCore ver.2's official mapping but not found in the dataset location /content/ShapeNetCore/ShapeNetCore: 03593526, 02946921, 04468005, 04225987, 04256520, 02773838, 02843684, 02954340, 03710193, 03790512, 03001627, 02880940, 04379243, 02828884, 02801938, 04330267, 02691156, 03207941, 02958343, 03991062, 03928116, 04460130, 04074963, 03797390, 04099429, 04530566, 02871439, 03948459, 04401088, 03325088, 03337140, 02933112, 03691459, 04554684, 04004475, 03636649, 02876657, 03759954, 02747177, 03624134, 03085013, 02818832, 04090263, 03938244, 03761084, 02924116, 03467517, 03513137, 03261776, 02942699
  warnings.warn(msg)


Total models loaded: 3891
